# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/dsECZ/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298168685276                   -0.85    5.6         
  2   -8.300168381933       -2.70       -1.25    1.0    105ms
  3   -8.300434062221       -3.58       -1.89    2.9    139ms
  4   -8.300462006494       -4.55       -2.77    1.9    149ms
  5   -8.300464286395       -5.64       -3.05    3.5    153ms
  6   -8.300464459379       -6.76       -3.22    2.2    133ms
  7   -8.300464550451       -7.04       -3.37    1.1    107ms
  8   -8.300464600129       -7.30       -3.52    3.8    180ms
  9   -8.300464632574       -7.49       -3.72    1.4    143ms
 10   -8.300464634248       -8.78       -3.76    1.0    108ms
 11   -8.300464641036       -8.17       -3.96    2.1    136ms
 12   -8.300464643980       -8.53      

In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67434793944                   -0.70    5.9         
  2   -16.67858653404       -2.37       -1.14    1.4    275ms
  3   -16.67920130255       -3.21       -1.87    3.2    339ms
  4   -16.67927592998       -4.13       -2.75    2.4    296ms
  5   -16.67928591537       -5.00       -3.16    5.5    430ms
  6   -16.67928620456       -6.54       -3.46    2.5    315ms
  7   -16.67928621869       -7.85       -3.94    1.6    258ms
  8   -16.67928622150       -8.55       -4.56    2.8    305ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.31777335379                   -0.56    6.5         
  2   -33.33439306246       -1.78       -1.00    2.0    1.04s
  3   -33.33596348339       -2.80       -1.71    4.8    1.32s
  4   -33.33616348970       -3.70       -2.63    3.4    1.13s
  5   -33.33690462789       -3.13       -2.53    8.9    1.76s
  6   -33.33692752006       -4.64       -2.73    2.8    1.02s
  7   -33.33694357182       -4.79       -3.62    2.6    1.13s
  8   -33.33694367578       -6.98       -3.71    3.8    1.27s
  9   -33.33694375478       -7.10       -4.00    1.1    894ms
 10   -33.33694378226       -7.56       -4.71    2.2    1.09s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298285344100                   -0.85    5.2         
  2   -8.300263703502       -2.70       -1.58    1.0   83.1ms
  3   -8.300437508972       -3.76       -2.58    3.2    119ms
  4   -8.300399594427   +   -4.42       -2.34    6.9    189ms
  5   -8.300463875524       -4.19       -3.33    1.0   86.5ms
  6   -8.300464491896       -6.21       -3.67    5.5    200ms
  7   -8.300464638842       -6.83       -4.33    1.6    112ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32589275245                   -0.56    7.2         
  2   -33.33285030477       -2.16       -1.28    1.1    819ms
  3   -33.31242538549   +   -1.69       -1.72    3.8    1.10s
  4   -29.77944966608   +    0.55       -0.78    7.1    1.70s
  5   -32.94177326149        0.50       -1.18    4.9    1.57s
  6   -33.32121834018       -0.42       -1.91    3.4    1.29s
  7   -32.99184550592   +   -0.48       -1.29    5.1    1.47s
  8   -33.33372532328       -0.47       -2.10    3.8    1.24s
  9   -33.33573550023       -2.70       -2.28    2.0    1.05s
 10   -33.33678230734       -2.98       -2.72    2.0    1.01s
 11   -33.33671746511   +   -4.19       -2.82    2.0    1.02s
 12   -33.33693928190       -3.65      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.